In [ ]:
import configobj
from crate import client
from fuzzywuzzy import process

from sklearn import linear_model
from sklearn import datasets, metrics
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import traceback

In [ ]:
def moneyline_from_implied_odds(p):
    if p < 0.5:
        return int(-1. * (100. * (p - 1.0)) / p)
    else:
        return int((100. * p) / (p - 1.0))
    
def implied_odds_from_moneyline(ml):

    if ml < 0.:
        return ( -1.0* ml)  / (( -1.0*ml )  + 100.)
    else:
        return 100. / ( ml + 100. )
    
def profit_from_odds(stake, ml):
    
    if ml > 0:
        return stake * (ml/100.)
    else:
        return stake / (-1.*ml/100.)

In [ ]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

sql = '''
SELECT * from mlb.pitchers
'''
df = pd.read_sql(sql, connection)
print df.head()

pitchers = df['Name'].values

sql = '''
SELECT
mlb.games."GameId" as "GameId", 
mlb.games."GameDate" as "GameDate", 
mlb.games."HomeTeam" as "HomeTeam", 
mlb.games."AwayTeam" as "AwayTeam",
mlb.games."HomeStarter" as "HomeStarter",
mlb.games."AwayStarter" as "AwayStarter",
mlb.results."AwayScore" as "AwayScore", 
mlb.results."HomeScore" as "HomeScore", 
mlb.lines."OverUnder" as "OverUnder",
mlb.lines."HomeLine" as "HomeLine",
mlb.lines."AwayLine" as "AwayLine"
FROM mlb.games, mlb.lines, mlb.results
WHERE mlb.games."GameId" = mlb.results."GameId"
AND mlb.games."GameId" = mlb.lines."GameId"
'''
df = pd.read_sql(sql, connection)

print df.head()

print len(df)

In [ ]:
#X = []
#y = []
dataset = []

nrows = len(df)

for row in df.iterrows():
    
    hometeam = str(row[1][2])
    awayteam = str(row[1][3])
    game_id = row[1][0]
    
    sql = '''SELECT "ERA" FROM mlb.pitching where "Team" = '%s' '''%hometeam
    cursor.execute(sql)
    home_era = cursor.fetchone()[0]
    
    sql = '''SELECT "ERA" FROM mlb.pitching where "Team" = '%s' '''%awayteam
    cursor.execute(sql)
    away_era = cursor.fetchone()[0]
    
    sql = '''SELECT "RAPG" FROM mlb.pitching where "Team" = '%s' '''%hometeam
    cursor.execute(sql)
    home_rapg = cursor.fetchone()[0]
    
    sql = '''SELECT "RAPG" FROM mlb.pitching where "Team" = '%s' '''%awayteam
    cursor.execute(sql)
    away_rapg = cursor.fetchone()[0]

    sql = '''SELECT "RPG" FROM mlb.batting where "Team" = '%s' '''%hometeam
    cursor.execute(sql)
    home_rpg = cursor.fetchone()[0]
    
    sql = '''SELECT "RPG" FROM mlb.batting where "Team" = '%s' '''%awayteam
    cursor.execute(sql)
    away_rpg = cursor.fetchone()[0]
    
    sql = '''SELECT "OBP" FROM mlb.batting where "Team" = '%s' '''%hometeam
    cursor.execute(sql)
    home_obp = cursor.fetchone()[0]
    
    sql = '''SELECT "OBP" FROM mlb.batting where "Team" = '%s' '''%awayteam
    cursor.execute(sql)
    away_obp = cursor.fetchone()[0]
    
    home_starter = row[1][4]
    away_starter = row[1][5]
    home_starter_1 = process.extractOne(home_starter, pitchers)[0]
    away_starter_1 = process.extractOne(away_starter, pitchers)[0]
    
    try:
        sql = '''SELECT "ERA","WLPct" FROM mlb.pitchers where "Name" like '%s' and "Team" = '%s' '''%(home_starter_1, hometeam)
        cursor.execute(sql)
        ret = cursor.fetchone()
        home_starter_era = ret[0]
        home_starter_wlpct = ret[1]
    except:
        print "Home Starter" + '\t' + home_starter + '\t' + home_starter_1 + '\t' + hometeam
        home_starter_era = home_era
        home_starter_wlpct = 0.5
    
    try:
        sql = '''SELECT "ERA","WLPct" FROM mlb.pitchers where "Name" like '%s' and "Team" = '%s' '''%(away_starter_1, awayteam)
        cursor.execute(sql)
        ret = cursor.fetchone()
        away_starter_era = ret[0]
        away_starter_wlpct = ret[1]
    except:
        print "Away Starter" + '\t' + away_starter + '\t' + away_starter_1 + '\t' + awayteam
        away_starter_era = away_era
        away_starter_wlpct = 0.5
    
    homescore = row[1][5]
    awayscore = row[1][4]
    
    if homescore > awayscore:
        homeresult = 1.0
        awayresult = 0.0
    if awayscore > homescore:
        homeresult = 0.0
        awayresult = 1.0
        
    #X.append(np.asarray([home_rpg, home_obp, away_era, away_starter_era]))
    #y.append(homeresult)
    
    dataset.append({'game id':game_id,
                    'home rpg':home_rpg,
                    'away rpg':away_rpg,
                    'home opb':home_obp,
                    'away obp':away_obp,
                    'home era':home_era,
                    'away era':away_era,
                    'home starter era':home_starter_era,
                    'away starter era':away_starter_era,
                    'result':homeresult})
    
    row = [game_id, home_rpg, away_rpg, home_obp, away_obp, home_era, away_era, home_starter_era, away_starter_era, homeresult]
    #print row
    query = '''INSERT INTO mlb.model ("GameId","HomeRPG", "AwayRPG", "HomeOBP", "AwayOBP", 
                                      "HomeERA", "AwayERA", "HomeStarterERA", "AwayStarterERA","Result") 
                                        VALUES (?,?,?,?,?,?,?,?,?,?)'''
    try:
        cursor.execute(query, row)
    except:
        print traceback.format_exc()

    
#X = np.asarray(X)
#y = np.asarray(y)
dataset = pd.DataFrame(dataset)
print dataset.head()

In [ ]:
print y_test

In [ ]:
query = '''SELECT "HomeRPG", "AwayRPG", "HomeOBP", "AwayOBP", 
          "HomeERA", "AwayERA", "HomeStarterERA", "AwayStarterERA","Result",
          "HomeTeam", "AwayTeam", "HomeStarter", "AwayStarter", "GameDate"
           FROM mlb.model, mlb.games
           WHERE mlb.model."GameId" = mlb.games."GameId" 
           AND "GameDate" < 20170000'''
cursor.execute(query)

X_train = []
y_train = []

for row in cursor:
    X_train.append(row[:8])
    y_train.append(row[8])
    
query = '''SELECT mlb.games."GameId", "HomeRPG", "AwayRPG", "HomeOBP", "AwayOBP", 
          "HomeERA", "AwayERA", "HomeStarterERA", "AwayStarterERA","Result",
          "HomeTeam", "AwayTeam", "HomeStarter", "AwayStarter", "GameDate"
           FROM mlb.model, mlb.games
           WHERE mlb.model."GameId" = mlb.games."GameId" 
           AND "GameDate" > 20170000'''
cursor.execute(query)

X_test = []
y_test = []
games = []

for row in cursor:
    games.append(row[0])
    X_test.append(row[1:9])
    y_test.append(row[9])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

# sc = StandardScaler()
# sc.fit(X_train)
# X_train_std = sc.transform(X_train)
# X_test_std = sc.transform(X_test)
# print X_train_std

# X_train = X[0:1500]
# y_train = y[0:1500]
# X_test = X[1500:]
# y_test = y[1500:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=10, 
                                random_state=1,
                                n_jobs=2)
forest.fit(X_train, y_train)

In [ ]:
y_pred = forest.predict(X_test)
y_true = y_test
print metrics.accuracy_score(y_true, y_pred)
print metrics.roc_auc_score(y_true, y_pred)
print len(y_test)

In [ ]:
wins = 0
losses = 0
balance = 1230

for g, X, y in zip(games,X_test,y_test):
    
    prediction = forest.predict(X).reshape(1, -1)[0]
    predict_proba =  forest.predict_proba(X).reshape(1, -1)[0]
    
    p_win = predict_proba[1]
    p_loss = predict_proba[0]
    
    sql = '''SELECT "HomeLine", "AwayLine" FROM mlb.lines where "GameId" = '%s' '''%g
    cursor.execute(sql)
    lines = cursor.fetchone()
    home_line = lines[0]
    home_odds = implied_odds_from_moneyline(home_line)
    away_line = lines[1]
    away_odds = implied_odds_from_moneyline(away_line)
    
    #choice = np.random.choice([0,1])
    
    if p_win > home_odds: # bet the home team
        choice = 1
    else:
        choice = 0
        
    if choice == 1 and y == 1.0:

        balance += profit_from_odds(1.0, home_line)
        print "Win " + str(profit_from_odds(1.0, home_line))
        wins += 1

    elif choice == 0 and y == 0.0:

        balance += profit_from_odds(1.0, away_line)
        print "Win " + str(profit_from_odds(1.0, away_line))
        wins += 1
    else:
        print "Lose 1"
        losses += 1
        balance -= 1
            
    print "Balance :" + str(balance)
        
print balance
print wins
print losses